In [ ]:
#process origin data from step 1

import pandas as pd
import numpy as np

def texts_to_corpus(temp_text):
    uni_words = np.unique(np.array(temp_text))
    corpus = []
    for uni_word in uni_words:
        corpus.append((uni_word,temp_text.count(uni_word)))
    return corpus

third_stage = pd.read_csv(r'\1Hz.csv')

# find the word with low frequency: time = 1

all_words = third_stage['words'].values
#print(len(np.unique(all_words)))
low_frequency_words = []
words_id = []
for i in range(len(all_words)):
    if list(all_words).count(all_words[i])<=1:
        words_id.append(all_words[i])
        print(third_stage.loc[i]['id'],third_stage.loc[i]['t'],all_words[i],list(all_words).count(all_words[i]))
        low_frequency_words.append([third_stage.loc[i]['id'],third_stage.loc[i]['t'],all_words[i],i])
#list(all_words).count(688)

unique_word = list(set(words_id))
print(unique_word,len(unique_word))

In [ ]:
#clean the data: remove the word id with low frequency

clean_data = third_stage
origin_words = third_stage['words'].values
for i in range(len(low_frequency_words)):
    temp_id = low_frequency_words[i][0]
    temp_t = low_frequency_words[i][1]
    temp_i = low_frequency_words[i][-1]
    print(temp_id,temp_t,temp_i)
    
    if temp_t-1>0 and clean_data.loc[temp_i-1]['words'] not in unique_word \
        and clean_data.loc[temp_i-1]['id'] == temp_id and clean_data.loc[temp_i-1]['t'] == clean_data.loc[temp_i]['t']-1:
        origin_words[temp_i]=clean_data.loc[temp_i-1]['words']
    elif temp_t-2>0 and clean_data.loc[temp_i-2]['words'] not in unique_word \
        and clean_data.loc[temp_i-2]['id'] == temp_id and clean_data.loc[temp_i-2]['t'] == clean_data.loc[temp_i]['t']-2:
        origin_words[temp_i]=clean_data.loc[temp_i-2]['words']
    else:
        origin_words[temp_i]=clean_data.loc[temp_i+1]['words']

clean_data['words'] = pd.DataFrame(np.array(origin_words))
print(clean_data)

clean_data.to_csv(r'\clean_1Hz(new).csv',index=None)

In [ ]:
import pandas as pd
import numpy as np

def texts_to_corpus(temp_text):
    uni_words = np.unique(np.array(temp_text))
    corpus = []
    for uni_word in uni_words:
        corpus.append((uni_word,temp_text.count(uni_word)))
    return corpus

def test2str(t_texts):
    str_text = []
    for t_text in t_texts:
        str_text.append(str(t_text))
    return str_text

third_stage = pd.read_csv(r'\clean_1Hz(new).csv')

#the texts are represented by each driver

ids = np.unique(third_stage['id'].to_numpy())
""
texts = []
for ID in ids:
    print(ID)
    single_data = third_stage[third_stage['id']==ID].reset_index().drop(columns = ['index'])
    temp_words = single_data['words'].values
    #print(temp_words)
    temp_text = []
    for temp_word in temp_words:
        temp_text.append(str(temp_word))
    #print(temp_text)
    texts.append(temp_text)


In [ ]:
# construct the dictionary, keep the words with high frequency

from gensim import corpora, models, similarities
#import jieba.posseg as jp, jieba
dictionary = corpora.Dictionary(texts)
#dictionary.filter_n_most_frequent(7)
corpus = [dictionary.doc2bow(words) for words in texts]
# dictionary = corpora.Dictionary.load(r'D:\科研\SNN\new_SNN_0222\dic_new.dict')
# corpus = [dictionary.doc2bow(words) for words in texts]
print(len(corpus))

In [ ]:
# save the dictionary

dictionary.save(r'\dic_new.dict')

In [ ]:
# use the tfidf to process the corpus

tfidf_model = models.TfidfModel(corpus)
corpus_tfidf = tfidf_model[corpus]
print(len(corpus_tfidf))

In [ ]:
from gensim.test.utils import common_corpus
from gensim.models.ldamodel import LdaModel
from gensim.test.utils import datapath
from gensim import models, corpora

ldamodel = LdaModel(corpus_tfidf,
            chunksize=2000,
            alpha='auto',
            eta='auto',
            decay=0.8,
            iterations=1200,
            num_topics=5,
            passes=20,
            eval_every=1)

temp_file = datapath(r"\auto_model_new_test.model")
ldamodel.save(temp_file)

# ldamodel = models.ldamodel.LdaModel.load(r"D:\科研\SNN\new_SNN_0222\auto_model.model")


In [ ]:
#test the effectiveness of ldamodel

test_cor = dictionary.doc2bow(['683','528','501','714','497','714','683'])
#test_cor = [words_to_cor(683),words_to_cor(528),words_to_cor(501),words_to_cor(714),words_to_cor(497),words_to_cor(714),words_to_cor(683)]
print(test_cor,ldamodel[tfidf_model[test_cor]])

test_cor = dictionary.doc2bow(['657','831','840','863','687','869','807'])
#test_cor = [words_to_cor(657),words_to_cor(831),words_to_cor(840),words_to_cor(863),words_to_cor(687),words_to_cor(869),words_to_cor(807)]
print(test_cor,ldamodel[tfidf_model[test_cor]])

test_cor = dictionary.doc2bow(['499','683','714','684','558','528','528'])
#test_cor = [words_to_cor(499),words_to_cor(683),words_to_cor(714),words_to_cor(684),words_to_cor(558),words_to_cor(528),words_to_cor(528)]
print(test_cor,ldamodel[tfidf_model[test_cor]])

test_cor = dictionary.doc2bow(['373','404','404','373','404','373','372'])
#test_cor = [words_to_cor(373),words_to_cor(404),words_to_cor(404),words_to_cor(373),words_to_cor(404),words_to_cor(373),words_to_cor(372)]
print(test_cor,ldamodel[tfidf_model[test_cor]])

#test_cor = [(528, 6), (558, 1)]
test_cor = dictionary.doc2bow(['528','528','528','528','528','528','558'])
#test_cor = [words_to_cor(373),words_to_cor(404),words_to_cor(404),words_to_cor(373),words_to_cor(404),words_to_cor(373),words_to_cor(372)]
print(test_cor,ldamodel[tfidf_model[test_cor]])

In [ ]:
def search_known(prediction):
    known_topics = []
    knowm_fea = []
    for i in range(len(prediction)):
        known_topics.append(prediction[i][0])
        knowm_fea.append(prediction[i][1])
    return known_topics,knowm_fea

def construct_feature(prediction,num_topic):
    known_topics,knowm_fea = search_known(prediction)
    #print(known_topics)
    temp_feature = []
    if len(known_topics)==num_topic:
        return knowm_fea
    for i in range(num_topic):
        completion = (1-sum(knowm_fea))/(num_topic-len(known_topics))
        if i in known_topics:
            indexs = known_topics.index(i)
            temp_feature.append(knowm_fea[indexs])
        else:
            temp_feature.append(completion)
    return temp_feature

def test2str(t_texts):
    str_text = []
    for t_text in t_texts:
        str_text.append(str(t_text))
    return str_text

In [ ]:
ids = np.unique(third_stage['id'].to_numpy())
total_times = 0
num_topic = 5
train_data = []
single_length = []

for ID in ids:
    print(ID)
    single_data = third_stage[third_stage['id']==ID].reset_index().drop(columns = ['index'])
    
    for i in range(len(single_data)-6):
        #for j in range(i,i+7):
        temp_text = [single_data.loc[i]['words'],single_data.loc[i+1]['words'],single_data.loc[i+2]['words'],
                    single_data.loc[i+3]['words'],single_data.loc[i+4]['words'],single_data.loc[i+5]['words'],
                    single_data.loc[i+6]['words']]
        
        tuple_corpus = dictionary.doc2bow(test2str(temp_text))
        temp_feature = construct_feature(ldamodel[tfidf_model[tuple_corpus]],num_topic)
        ID = single_data.loc[i+6]['id']
        t = single_data.loc[i+6]['t']
        corpus = dictionary.doc2bow([str(single_data.loc[i+6]['words'])])
#         his_text = [single_data.loc[i+1]['words'],single_data.loc[i+2]['words'],
#                     single_data.loc[i+3]['words'],single_data.loc[i+4]['words'],single_data.loc[i+5]['words'],
#                     single_data.loc[i+6]['words']]
        train_data.append([ID,t,len(single_data)-6,corpus,temp_feature,int(single_data.loc[i+6]['words'])])
    total_times += len(single_data)-6
    single_length.append(len(single_data)-6)

In [ ]:
def pad_seq(temp_train):
    np_train = np.array(temp_train)
    if np_train.shape[0]>15:
        print('error')
        return np_train
    elif np_train.shape[0]==15:
        return np_train
    else:
        loss_len = 15-np_train.shape[0]
        loss_train = np.zeros([loss_len,5])
        return np.concatenate([loss_train,np_train],axis=0)
    
ids = np.unique(third_stage['id'].to_numpy())
flag = 0
minimum = 4
maximum = 14
all_train = []
train_speed = []
target_speed = []
np_all_train = np.zeros([0,15,5])

for ID in ids:
    print(ID)
    single_data = third_stage[third_stage['id']==ID].reset_index().drop(columns = ['index'])
    length = single_length[int(ID)-1]
    temp_data = train_data[flag:flag+length]
    flag += length
    if length<4+1:
        continue
    for i in range(length-minimum+1):
        end_num = min(length-1,i+maximum)
        #print(i,end_num)
        for j in range(i+minimum,end_num+1):
            next_speed = temp_data[j][5]
            current_speed = temp_data[j-1][5]
            train_speed.append(next_speed)
            target_speed.append(current_speed)
            temp_train = []
            for k in range(i,j+1):
                temp_train.append(temp_data[k][4])
            np_temp_train = pad_seq(temp_train).reshape([-1,pad_seq(temp_train).shape[0],pad_seq(temp_train).shape[1]])
            all_train.append(temp_train)
            np_all_train = np.concatenate([np_all_train,np_temp_train],axis=0)

In [ ]:
np_train_speed = np.array(train_speed).reshape(-1,1)
np_target_speed = np.array(target_speed).reshape(-1,1)
print(np_all_train.shape,np_train_speed.shape,np_target_speed.shape)

np.save(r'\np_all_train_new.npy',np_all_train)
np.save(r'\np_train_speed_new.npy',np_train_speed)
np.save(r'\np_target_speed_new.npy',np_target_speed)